<div class="alert alert-block alert-success">
    <h1 align="center">Machine Learning in Python</h1>
    <h3 align="center">Artificial Neural Network</h3>
    <h4 align="center"><a href="https://github.com/AliBinary">Ali Ghanbari</a></h5>
</div>

### Using PyTorch - Artificial Neural Network

## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\AliBinary\anaconda3\envs\ds_env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\AliBinary\anaconda3\envs\ds_env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\AliBinary\anaconda3\envs\ds_env\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start(

### Check PyTorch version and GPU

In [3]:
print("PyTorch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

PyTorch version: 2.2.2+cu121
Using device: cuda


## 2️⃣ Load Dataset

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')

### Features and labels

In [5]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

## 3️⃣ Encode Categorical Data
### Gender

In [6]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

## Geography (OneHot)

In [7]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

## 4️⃣ Split Dataset

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## 5️⃣ Feature Scaling

In [9]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Convert to PyTorch tensors

In [10]:
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

## 6️⃣ Build ANN Model

In [11]:
class ANNModel(nn.Module):
    def __init__(self, input_dim):
        super(ANNModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 6)
        self.layer2 = nn.Linear(6, 6)
        self.output = nn.Linear(6, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

input_dim = X_train.shape[1]
model = ANNModel(input_dim).to(device)

## 7️⃣ Compile Model

In [12]:
criterion = nn.BCELoss()           # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 8️⃣ Train Model

In [13]:
epochs = 100
batch_size = 32

for epoch in range(epochs):
    permutation = torch.randperm(X_train.size()[0])
    
    for i in range(0, X_train.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train[indices], y_train[indices]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.3934
Epoch [20/100], Loss: 0.4206
Epoch [30/100], Loss: 0.4122
Epoch [40/100], Loss: 0.3999
Epoch [50/100], Loss: 0.4140
Epoch [60/100], Loss: 0.3469
Epoch [70/100], Loss: 0.3175
Epoch [80/100], Loss: 0.4278
Epoch [90/100], Loss: 0.3685
Epoch [100/100], Loss: 0.3010


## 9️⃣ Evaluate Model

In [15]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_class = (y_pred > 0.5).float()

# Confusion matrix manually
tp = ((y_pred_class == 1) & (y_test == 1)).sum().item()
tn = ((y_pred_class == 0) & (y_test == 0)).sum().item()
fp = ((y_pred_class == 1) & (y_test == 0)).sum().item()
fn = ((y_pred_class == 0) & (y_test == 1)).sum().item()

print("Confusion Matrix:")
print([[tn, fp],
       [fn, tp]])

accuracy = (tp + tn) / (tp + tn + fp + fn)
print("Accuracy:", accuracy)

Confusion Matrix:
[[1539, 56], [264, 141]]
Accuracy: 0.84


## 1️⃣0️⃣ Predict Single Customer
## Customer Info (France, 600, Male, 40, 3, 60000, 2, Yes, Yes, 50000)

In [17]:
new_customer = np.array([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])
new_customer_scaled = sc.transform(new_customer)
new_customer_tensor = torch.tensor(new_customer_scaled, dtype=torch.float32).to(device)

with torch.no_grad():
    prediction = model(new_customer_tensor)
    print("Will the customer leave?", prediction.item() > 0.5)

Will the customer leave? False
